# Bin Packing Problem

### Data [Falkenauer, 1996]

In [3]:
import os
import pandas as pd
import numpy as np
from gurobipy import *
from random import *
from time import time

bpp_data_U = []
addr = r'C:\Users\Samuel\OneDrive\ITA\2020-2\PO-201\projeto\Falkenauer\Falkenauer U'

for filename in os.listdir(addr):
    if filename[:-5].endswith('0'):
        dic = {'instance_name':'', 'c': 0, 'w':[]}
        dic['instance_name'] = filename[:-4]
        fileaddr = os.path.join(addr, filename)
        with open(fileaddr, 'r') as file:
            n = file.readline()
            dic['c'] = int(file.readline())
            dic['w'] = [int(number) for number in file.readlines()]
        bpp_data_U.append(dic)     
    
def lb(c, w):
    return int(math.ceil(sum(w) / c))

df = pd.DataFrame(bpp_data_U, columns=['instance_name', 'c', 'w'])
df['n'] = df['w'].apply(len)
df['wmin'] = df['w'].apply(min)
df['wmax'] = df['w'].apply(max)
df['lb'] = df.apply(lambda x: lb(x['c'], x['w']), axis=1)
display(df)


,instance_name,c,w,n,wmin,wmax,lb
0,Falkenauer_u1000_00,150,"[100, 100, 100, 100, 100, 100, 100, 100, 100, ...",1000,20,100,399
1,Falkenauer_u1000_04,150,"[100, 100, 100, 100, 100, 100, 100, 100, 100, ...",1000,20,100,397
2,Falkenauer_u120_00,150,"[98, 98, 98, 96, 96, 94, 93, 93, 92, 91, 91, 9...",120,20,98,48
3,Falkenauer_u120_04,150,"[99, 99, 98, 98, 97, 97, 96, 95, 92, 92, 92, 9...",120,20,99,50
4,Falkenauer_u250_00,150,"[100, 100, 100, 99, 99, 98, 98, 98, 98, 98, 98...",250,20,100,99
5,Falkenauer_u250_04,150,"[100, 100, 99, 98, 98, 98, 97, 97, 97, 96, 95,...",250,20,100,101
6,Falkenauer_u500_00,150,"[100, 100, 100, 100, 100, 100, 99, 99, 99, 98,...",500,20,100,198
7,Falkenauer_u500_04,150,"[100, 100, 100, 100, 100, 100, 100, 100, 100, ...",500,21,100,206


In [4]:
bpp_data_T = []
addr = r'C:\Users\Samuel\OneDrive\ITA\2020-2\PO-201\projeto\Falkenauer\Falkenauer_T'

for filename in os.listdir(addr):
    if filename[:-5].endswith('0'):
        dic = {'instance_name':'', 'c': 0, 'w':[]}
        dic['instance_name'] = filename[:-4]
        fileaddr = os.path.join(addr, filename)
        with open(fileaddr, 'r') as file:
            n = file.readline()
            dic['c'] = int(file.readline())
            dic['w'] = [int(number) for number in file.readlines()]
        bpp_data_T.append(dic)     

df = pd.DataFrame(bpp_data_T, columns=['instance_name', 'c', 'w'])
df['n'] = df['w'].apply(len)
df['wmin'] = df['w'].apply(min)
df['wmax'] = df['w'].apply(max)
df['lb'] = df.apply(lambda x: lb(x['c'], x['w']), axis=1)
display(df)

,instance_name,c,w,n,wmin,wmax,lb
0,Falkenauer_t120_00,1000,"[497, 497, 495, 485, 480, 478, 474, 473, 472, ...",120,250,497,40
1,Falkenauer_t120_04,1000,"[499, 497, 491, 488, 484, 484, 483, 481, 480, ...",120,250,499,40
2,Falkenauer_t249_00,1000,"[498, 497, 497, 497, 496, 495, 495, 492, 491, ...",249,250,498,83
3,Falkenauer_t249_04,1000,"[499, 498, 498, 498, 498, 498, 496, 488, 486, ...",249,250,499,83
4,Falkenauer_t501_00,1000,"[498, 498, 498, 497, 497, 497, 496, 496, 495, ...",501,250,498,167
5,Falkenauer_t501_04,1000,"[499, 499, 498, 498, 495, 493, 493, 491, 490, ...",501,250,499,167
6,Falkenauer_t60_00,1000,"[495, 474, 473, 472, 466, 450, 445, 444, 439, ...",60,251,495,20
7,Falkenauer_t60_04,1000,"[498, 496, 494, 491, 478, 470, 455, 434, 428, ...",60,250,498,20


## Solver Model

In [6]:
def model_bpp(c, w, UB=None, bin_for_item=None, LogToConsole=False, TimeLimit=300):
    n = len(w)
    LB = lb(c, w)
    if UB == None:
        UB = n
    if LogToConsole:
        print('c =', c, '| n =', n, '| LB =', LB, '| UB =', UB)
    model = Model()
    model.params.LogToConsole = LogToConsole
    model.params.TimeLimit = TimeLimit # seconds
    x = model.addVars(n, UB, vtype=GRB.BINARY)
    y = model.addVars(UB, vtype=GRB.BINARY)
    model.setObjective(quicksum(y[j] for j in range(UB)), GRB.MINIMIZE) # minimize the number of bins used
    model.addConstrs(quicksum(x[i,j] for j in range(UB)) == 1 for i in range(n)) # each item in exactly one bin
    model.addConstrs(quicksum(w[i] * x[i,j] for i in range(n)) <= c * y[j] for j in range(UB))
                                                                  # limit total weight in each bin; also, link $x_{ij}$ with $y_j$
    if bin_for_item != None:
        for i in range(n):
            x[i, bin_for_item[i]].start = 1
    model.optimize()
    bin_for_item = [-1 for i in range(n)]
    for i in range(n):
        for j in range(UB):
            if x[i,j].X > 0.5:
                bin_for_item[i] = j
    return model.ObjVal, model.ObjBound, bin_for_item

## Heuristic: First-Fit-Decreasing

In [7]:
def heur_FFD(c, w):
    n = len(w)
    order = sorted([i for i in range(n)], key=lambda i: w[i], reverse=True) # sort by decreasing weights
    bin_for_item = [-1 for i in range(n)]
    bin_space = []
    for i in order:
        for j in range(len(bin_space)): # pack in the first bin in which the item fits
            if w[i] <= bin_space[j]:
                bin_for_item[i] = j
                bin_space[j] -= w[i]
                break
        if bin_for_item[i] < 0: # if no bin can accomodate the item, open a new bin
            j = len(bin_space)
            bin_for_item[i] = j
            bin_space.append(c - w[i])
    n_bins = len(bin_space)
    return n_bins, bin_for_item, bin_space

## Meta-Heuristic: Simulated Annealing

### Neighborhood method

In [8]:
def neighb(s, w, c):
    n, item_0, space_0 = s
    item = item_0.copy()
    space = space_0.copy()
    m = len(item)
    while True:
        # transfers item b to bin a
        a = randint(0, n-1)
        b = randint(0, m-1)
        if space[a] >= w[b] and space[a] < c: 
            space[item[b]] += w[b]
            space[a] -= w[b]
            item[b] = a
            break
        # swap items a and b
        a = randint(0, m-1) 
        if space[item[a]] + w[a] >= w[b] and space[item[b]] + w[b] >= w[a]:
            space[item[a]] += w[a] - w[b]
            space[item[b]] += w[b] - w[a]
            item[a], item[b] = item[b], item[a]
            break
    return n, item, space

### Objective function

#### $f = \sum_{i=1}^{n}\left(\sum_{k_i}w_j\right)^2$, with $k_i$ the set of items in bin $i$

In [9]:
def obj(s, w):
    n, item, space = s
    m = len(item)
    total = 0
    for i in range(n):
        weight = 0
        for j in range(m):
            if item[j] == i:
                weight += w[j]
        total -= weight**2  # negative weights (minimization problem)
    return total
            

### Simulated Annealing base algorithm

In [10]:
def SA(alpha, SAmax, Ti, Tf, s, w, c):
    best = s
    i = 0
    T = Ti
    while T > Tf:
        while i < SAmax:
            i += 1
            r = neighb(s, w, c)
            delta = obj(r, w) - obj(s, w)
            if delta <= 0: 
                s = r
                if obj(r, w) < obj(best, w):
                    best = r
            else:
                x = random()
                y = np.exp(-delta/T)
                if x < y:
                    s = r
        T = alpha*T
        i = 0
        
    n, item, space = best
    for i in range(n):  # number of bins update
        if space[i] == c:
            n -= 1
    return n, item, space   

### Modified algorithm to display solutions while running

#### Auxiliary function

In [11]:
def n_bins_updated(s0, s):
    n0, item0, space0 = s0
    n, item, space = s
    for i in range(n0):
        if space0[i] == c:
            n0 -= 1
        if space[i] == c:
            n -= 1
    if n<n0:
        return True, n
    return False, n

In [12]:
def SA_log(alpha, SAmax, Ti, Tf, s, w, c, t_i):
    best = s
    i = 0
    T = Ti
    while T > Tf:
        while i < SAmax:
            i += 1
            r = neighb(s, w, c)
            delta = obj(r, w) - obj(s, w)
            if delta <= 0: 
                s = r
                if obj(r, w) < obj(best, w):
                    chk, m = n_bins_updated(best, r)
                    if chk:    
                        t_f = time()
                        t = round(t_f-t_i, 2)
                        print(t, m)
                    best = r
            else:
                x = random()
                y = np.exp(-delta/T)
                if x < y:
                    s = r
        T = alpha*T
        i = 0
    
    n, item, space = best
    for i in range(n):  # number of bins update
        if space[i] == c:
            n -= 1
    return n, item, space  

## Instances

### FFD+Gurobi

#### Class U

In [8]:
print('idx', 'lb', 'heur', 'solv')
for idx, bpp_data in enumerate(bpp_data_U):
        c, w = bpp_data['c'], bpp_data['w']
        s_0 = heur_FFD(c, w)
        n_bins, n_bins_lb, bin_for_item = model_bpp(c, w, s_0[0], s_0[1], TimeLimit=300)
        print(idx, lb(c,w), s_0[0], n_bins)

idx lb heur solv
Using license file C:\Users\Samuel\gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-20
0 399 403 403.0
1 397 402 402.0
2 48 49 48.0
3 50 50 50.0
4 99 100 100.0
5 101 102 102.0
6 198 201 200.0
7 206 209 208.0


#### Class T

In [9]:
print('idx', 'lb', 'heur', 'solv')
for idx, bpp_data in enumerate(bpp_data_T):
        c, w = bpp_data['c'], bpp_data['w']
        s_0 = heur_FFD(c, w)
        n_bins, n_bins_lb, bin_for_item = model_bpp(c, w, s_0[0], s_0[1], TimeLimit=300)
        print(idx, lb(c,w), s_0[0], n_bins)

idx lb heur solv
0 40 45 41.0
1 40 46 41.0
2 83 94 85.0
3 83 95 85.0
4 167 190 170.0
5 167 191 169.0
6 20 23 20.0
7 20 24 20.0


#### Track

t249_04

In [6]:
for idx, bpp_data in enumerate(bpp_data_T):
    if idx == 3:
        c, w = bpp_data['c'], bpp_data['w']
        s_0 = heur_FFD(c, w)
        n_bins, n_bins_lb, bin_for_item = model_bpp(c, w, s_0[0], s_0[1], LogToConsole=True, TimeLimit=1000)

c = 1000 | n = 249 | LB = 83 | UB = 95
Using license file C:\Users\Samuel\gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-20
Parameter LogToConsole unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter TimeLimit to 1000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 344 rows, 23750 columns and 47405 nonzeros
Model fingerprint: 0xd03fe74b
Variable types: 0 continuous, 23750 integer (23750 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 95 (0.09s)
Loaded user MIP start with objective 95

Presolve time: 0.14s
Presolved: 344 rows, 23750 columns, 47405 nonzeros
Variable types: 0 continuous, 23750 integer (23750 binary)

Ro

u_500_04

In [20]:
for idx, bpp_data in enumerate(bpp_data_U):
    if idx == 7:
        c, w = bpp_data['c'], bpp_data['w']
        s_0 = heur_FFD(c, w)
        n_bins, n_bins_lb, bin_for_item = model_bpp(c, w, s_0[0], s_0[1], LogToConsole=True, TimeLimit=400)

c = 150 | n = 500 | LB = 206 | UB = 209
Using license file C:\Users\Samuel\gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-20
Parameter LogToConsole unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter TimeLimit to 400.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 709 rows, 104709 columns and 209209 nonzeros
Model fingerprint: 0x858a609e
Variable types: 0 continuous, 104709 integer (104709 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 209 (0.21s)
Loaded user MIP start with objective 209

Presolve time: 0.55s
Presolved: 709 rows, 104709 columns, 209209 nonzeros
Variable types: 0 continuous, 104709 integer (104709 b

### FFD+SA

#### Class U

In [17]:
# metaheuristic parameters
alpha = 0.95
SAmax = 100
Ti = 100
Tf = 0.1
print('alpha =', alpha, 'SAmax =', SAmax, 'Ti =', Ti, 'Tf =', Tf)
print('idx', 'lb', 'SA', 'time')
for idx, bpp_data in enumerate(bpp_data_U):
        c, w = bpp_data['c'], bpp_data['w']
        s_0 = heur_FFD(c, w)
        t_i = time()
        s = SA(alpha, SAmax, Ti, Tf, s_0, w, c)
        t_f = time()
        print(idx, lb(c,w), s[0], t_f-t_i)

alpha = 0.95 SAmax = 100 Ti = 100 Tf = 0.1
idx lb SA time
0 399 401 1551.2771146297455
1 397 400 1595.6033930778503
2 48 48 23.36166501045227
3 50 50 23.082386255264282
4 99 100 91.002596616745
5 101 101 106.73689842224121
6 198 199 380.7764298915863
7 206 207 387.8392231464386


#### Class T

In [14]:
# metaheuristic parameters
alpha = 0.95
SAmax = 100
Ti = 100
Tf = 0.1
print('alpha =', alpha, 'SAmax =', SAmax, 'Ti =', Ti, 'Tf =', Tf)
print('idx', 'lb', 'SA', 'time')
for idx, bpp_data in enumerate(bpp_data_T):
        c, w = bpp_data['c'], bpp_data['w']
        s_0 = heur_FFD(c, w)
        t_i = time()
        s = SA(alpha, SAmax, Ti, Tf, s_0, w, c)
        t_f = time()
        print(idx, lb(c,w), s[0], t_f-t_i)

alpha = 0.95 SAmax = 100 Ti = 100 Tf = 0.1
idx lb SA time
0 40 41 19.570784091949463
1 40 41 20.310540914535522
2 83 84 79.34288763999939
3 83 84 81.95137047767639
4 167 168 324.04775381088257
5 167 168 292.7296242713928
6 20 21 5.309537887573242
7 20 21 5.424614667892456


#### Track

t249_04

In [18]:
# metaheuristic parameters
alpha = 0.95
SAmax = 100
Ti = 100
Tf = 0.1
for idx, bpp_data in enumerate(bpp_data_T):
    if idx == 3:
        c, w = bpp_data['c'], bpp_data['w']
        s_0 = heur_FFD(c, w)
        t_i = time()
        s = SA_log(alpha, SAmax, Ti, Tf, s_0, w, c, t_i)
        t_f = time()
        print(t_f-t_i, s[0])

0.97 94
2.41 93
2.59 92
2.95 91
4.74 90
5.35 89
5.7 88
7.62 87
11.32 86
12.64 85
19.77 84
74.96133089065552 84


u_500_04

In [19]:
# metaheuristic parameters
alpha = 0.95
SAmax = 100
Ti = 100
Tf = 0.1
for idx, bpp_data in enumerate(bpp_data_U):
    if idx == 7:
        c, w = bpp_data['c'], bpp_data['w']
        s_0 = heur_FFD(c, w)
        t_i = time()
        s = SA_log(alpha, SAmax, Ti, Tf, s_0, w, c, t_i)
        t_f = time()
        print(round(t_f-t_i,2), s[0])

146.19 208
181.65 207
369.48 206
371.69 206
